In [3]:
import pandas as pd
import numpy as np
import json
import warnings
from evaluator import *

# Import data

In [4]:
# file path to data folder
path = './Output'

Metadata (subjectID etc.)

In [5]:
with open(path + '/metadata_stat.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

In [6]:
subjects = [meta['subjectID'] for meta in metadata]

# Model

In [7]:
class MajorityVoter:

    def __init__(self, path, variable):
        self.SEED = 42
        self.path = path
        assert variable in (0, 1)
        self.variable = variable
        self.majority_vote = None

    def load_data(self, indices):
        # load shape
        N = len(indices)
        N_FEATURES = np.load(self.path + '/feature_vector_stat0.npy').shape[0]

        # init
        X = np.empty((N, N_FEATURES))
        y = np.empty(N)

        # load individual datapoints
        for i, index in enumerate(indices):
            X[i, ] = np.load(path + f'/feature_vector_stat{index}.npy', allow_pickle=True)
            y[i, ] = np.load(path + f'/labels_stat{index}.npy', allow_pickle=True)[self.variable]

        return X, y

    def fit(self, train_indices):
        # load data
        _, y_train = self.load_data(train_indices)

        self.majority_vote = np.round(np.nanmean(y_train, axis=0))

    def predict(self, test_indices):
        n_predictions = len(test_indices)

        # predict
        y_pred = np.array([self.majority_vote for _ in range(n_predictions)])

        return y_pred

# CV

In [8]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

    for variable in (0, 1): # phF, MF
        model = MajorityVoter(path, variable=0)

        scores_strat_group_k_fold = stratified_group_k_fold(path=path,
                                                            groups=subjects,
                                                            model=model,
                                                            folds=5,
                                                            images=False,
                                                            verbose=True,
                                                            variable=variable)

        scores_strat_k_fold = stratified_k_fold(path=path,
                                                groups=subjects,
                                                model=model,
                                                folds=5,
                                                images=False,
                                                verbose=True,
                                                variable=variable)

        scores_loso = leave_one_subject_out(path=path,
                                            groups=subjects,
                                            model=model,
                                            images=False,
                                            verbose=True,
                                            variable=variable)

Starting stratified group 5-fold for physical fatigue


 Fold 5 F1: 0.382010582010582: 100%|██████████| 5/5 [00:00<00:00, 16.50it/s] 


Performance model:
 accuracy: 0.733 +- 0.151 

 balanced_accuracy: 0.5 +- 0.0 

 f1: 0.629 +- 0.203 

 recall: 0.733 +- 0.151 

 precision: 0.56 +- 0.228 

Starting stratified 5-fold for physical fatigue


 Fold 5 F1: 0.6589446589446589: 100%|██████████| 5/5 [00:00<00:00, 18.33it/s]


Performance model:
 accuracy: 0.757 +- 0.006 

 balanced_accuracy: 0.5 +- 0.0 

 f1: 0.653 +- 0.008 

 recall: 0.757 +- 0.006 

 precision: 0.573 +- 0.009 

Starting leave-one-subject-out for physical fatigue


...

Performance model:
 accuracy: 0.661 +- 0.128 

 balanced_accuracy: 0.5 +- 0.0 

 f1: 0.533 +- 0.165 

 recall: 0.661 +- 0.128 

 precision: 0.454 +- 0.176 

Starting stratified 5-fold for mental fatigue


 Fold 5 F1: 0.5743954902833408: 100%|██████████| 5/5 [00:00<00:00, 19.08it/s]


Performance model:
 accuracy: 0.694 +- 0.005 

 balanced_accuracy: 0.5 +- 0.0 

 f1: 0.569 +- 0.007 

 recall: 0.694 +- 0.005 

 precision: 0.482 +- 0.007 

Starting leave-one-subject-out for mental fatigue


 Fold 21 F1: 0.3333333333333333: 100%|██████████| 21/21 [00:01<00:00, 16.28it/s] 

Performance model:
 accuracy: 0.573 +- 0.318 

 balanced_accuracy: 0.5 +- 0.267 

 f1: 0.48 +- 0.333 

 recall: 0.573 +- 0.318 

 precision: 0.429 +- 0.337 

